In [1]:
#code from Roberto, Nov.29, modify and see how long it takes to run 
from keras.applications import VGG16
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread  #installed skimage - image processing library for sklearn
from skimage.transform import resize 
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model
from sklearn.metrics.pairwise import euclidean_distances

%matplotlib inline

Using TensorFlow backend.


In [2]:
#my imports
from keras.preprocessing import image
from keras_applications import imagenet_utils
#see doc for built-in functions https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py#L157
from keras.applications.vgg16 import VGG16, preprocess_input

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

import os
import time
import sys
import zipfile

In [3]:
#unzip:
#with zipfile.ZipFile('image_data/allre_images.zip', 'r') as zip_ref:
#    zip_ref.extractall('image_data/allrecipes_photos') #only needed to run this once to unzip 


In [4]:
test = !ls image_data/allrecipes_photos/allre_images/userphotos/250x250
len(test)
 #was 35k before, upload zipped file again??

35299

Get the list of images

In [5]:
#shoe_list = !ls ZShoes/Images/

#! - runs command line
#!ls project5data/2017_140k/recipe_photos/bbc_photos/pages-photos

#bbc_list = !ls project5data/2017_140k/recipe_photos/bbc_photos/pages-photos  
    #local directory, doesn't work on aws? modified:
    
alr_list = !ls image_data/allrecipes_photos/allre_images/userphotos/250x250

In [6]:
len(alr_list) #35300 images in allrecipes 250x250 dataset, ~60k in 560x315 dataset
#len only 3001? normal on local data

35299

In [7]:
alr_list[10] #not labelled

'1000675.jpg'

In [8]:
type(alr_list)

IPython.utils.text.SList

In [9]:
#shoe_number = [int(x.split('.')[0]) for x in shoe_list]
#shoe_number.sort()

In [10]:
#shoe_number[:5] # may need numbers later to grab every 10 images or so

Setup the VGG16 Model

In [11]:
# load VGG16
#Get back the convolutional part of a VGG network trained on ImageNet
model = VGG16(weights='imagenet', include_top=True)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
def image_pre_process(img_file):
    img = imread(img_file)  
    img = resize(img, (224, 224), preserve_range=True).astype(np.float32) #resize image
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img) #keras preprocessing function
    return img

In [14]:
model2 = Model(input=model.input, output=model.get_layer('fc2').output) # gets the output in fc2

/Library/anaconda/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """Entry point for launching an IPython kernel.


In [15]:
# def get_images(directory):
#     '''stores all images in directory into a list'''
#     img_lst = []
#     for entry in os.scandir(directory): #loop through files in directory, path is file path of folder containing images
#         if entry.path == directory + '/.DS_Store':
#             continue  #avoid reading mac os's .DS_Store as an image
#         else:
#             img_lst.append(entry.path)
#     return img_lst

Function that can get the data in chunks

In [16]:
# def process_data(start, end):
#     out1 = []
#     for i in range(start,end):
#         print(i)
#         name = 'ZShoes/Images/' + str(i) + '.jpg'
#         im = image_pre_process(name)
#         out = model2.predict(im)
#         out1.append(out[0])
#     return out1

#looping through images in directory
# def get_images(directory):
#     '''stores all images in directory into a list'''
#     img_lst = []
#     for entry in os.scandir(directory): #loop through files in directory, path is file path of folder containing images
#         if entry.path == directory + '/.DS_Store':
#             continue  #avoid reading mac os's .DS_Store as an image
#         else:
#             img_lst.append(entry.path)
#     return img_lst

In [17]:
#def process_data(start, end, directory):
def process_data(imglist,directory):
    out1 = []
    print(len(imglist))
    for file in imglist:
        #print(file)
        filepath = directory + file
        #print(filepath)
        im = image_pre_process(filepath)
        out = model2.predict(im)
        out1.append(out[0])
        if len(out1) %100 == 0:
            print(str(len(out1)) + ' out of ' + str(len(imglist)) + ' done')
    return out1

In [18]:
alrpath = 'image_data/allrecipes_photos/allre_images/userphotos/250x250/' #new path for aws, add'/'

In [19]:
out1 = process_data(alr_list[:100], alrpath)
res = pd.DataFrame(out1)

#still 1min for 100 images like in bbc case, makes sense (but just as slow as local one?)

#Nov30, zipped and uploaded data again, on aws, 1min 34s, public wifi - slower than at Metis
#home, aws, 2m for first 100? Internet speed?

100
100 out of 100 done


In [26]:
out1 = process_data(alr_list[:1001], alrpath)
res = pd.DataFrame(out1)
#for bbc case, full list of 2226 images, 35mins, still appending df though
#*optimize code to add each row of df to csv first?

#11min33s for 1000 images, reasonable (smaller chunk, faster)

1001
100 out of 1001 done
200 out of 1001 done
300 out of 1001 done
400 out of 1001 done
500 out of 1001 done
600 out of 1001 done
700 out of 1001 done
800 out of 1001 done
900 out of 1001 done
1000 out of 1001 done


In [27]:
res.to_csv('allreFeatures1001.csv') #took 6s to write one big dataframe to csv?

In [29]:
out2 = process_data(alr_list[1001:2001], alrpath)
res = pd.DataFrame(out2)
#11min47s

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [30]:
res.to_csv('allreFeatures2001.csv')

In [31]:
out2 = process_data(alr_list[2001:3001], alrpath)
res = pd.DataFrame(out2)


1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [33]:
res.to_csv('allreFeatures3001.csv')

---

In [ ]:
#append every 1000 images to large df? or can just load every 1000 (small csv into df with function? - depending on how to use data)

In [14]:
#Nov 30 evening, start here, append to background while analyzing bbc /allre subset data (can do both on aws?)
#technically can do this locally also, not much slower*
out6 = process_data(alr_list[5001:6001], alrpath)
res = pd.DataFrame(out6)

#a lot slower, why? earlier 11min; aws shouldn't be affected by local changes

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [15]:
#append processing results to csv:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [16]:
out7 = process_data(alr_list[6001:7001], alrpath)
res = pd.DataFrame(out7)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [17]:
#append processing results to csv:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [18]:
out8 = process_data(alr_list[7001:8001], alrpath)
res = pd.DataFrame(out8)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [19]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False) #better way to automate this?

In [20]:
out9 = process_data(alr_list[8001:9001], alrpath)
res = pd.DataFrame(out9)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [21]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [22]:
out10 = process_data(alr_list[9001:10001], alrpath)
res = pd.DataFrame(out10)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [23]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [24]:
out11 = process_data(alr_list[10001:11001], alrpath)
res = pd.DataFrame(out11)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [25]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [26]:
out12 = process_data(alr_list[11001:12001], alrpath)
res = pd.DataFrame(out12)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [27]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)  #1/3 of large dataset

In [28]:
out13 = process_data(alr_list[12001:13001], alrpath)
res = pd.DataFrame(out13)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [29]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [ ]:
###
#Nov 30 8:30pm start again from 14k, 0-13k csv already 249MB

In [20]:
out14 = process_data(alr_list[13001:14001], alrpath) #stuck on this for 30mins without printing anything - why?
res = pd.DataFrame(out14)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [21]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [22]:
out15 = process_data(alr_list[14001:15001], alrpath)
res = pd.DataFrame(out15)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [23]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [24]:
out16 = process_data(alr_list[15001:16001], alrpath)
res = pd.DataFrame(out16)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [25]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [26]:
out17 = process_data(alr_list[16001:17001], alrpath)
res = pd.DataFrame(out17)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [27]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False) #~1/2 of 250x250 dataset, csv 375MB

In [ ]:
#Dec1, 2nd half

In [19]:
out18 = process_data(alr_list[17001:18001], alrpath)
res = pd.DataFrame(out18) 

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [20]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [21]:
out19 = process_data(alr_list[18001:19001], alrpath)
res = pd.DataFrame(out19)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [22]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [23]:
out20 = process_data(alr_list[19001:20001], alrpath)
res = pd.DataFrame(out20)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [24]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [25]:
out21 = process_data(alr_list[20001:21001], alrpath)
res = pd.DataFrame(out21)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [26]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [27]:
out22 = process_data(alr_list[21001:22001], alrpath)
res = pd.DataFrame(out22)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [28]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [29]:
out23 = process_data(alr_list[22001:23001], alrpath)
res = pd.DataFrame(out23)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [30]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [31]:
out24 = process_data(alr_list[23001:24001], alrpath)
res = pd.DataFrame(out24)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [32]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [33]:
out25 = process_data(alr_list[24001:25001], alrpath)
res = pd.DataFrame(out25)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [34]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [35]:
out26 = process_data(alr_list[25001:26001], alrpath)
res = pd.DataFrame(out26)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [36]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [37]:
out27 = process_data(alr_list[26001:27001], alrpath)
res = pd.DataFrame(out27)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [38]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [39]:
out28 = process_data(alr_list[27001:28001], alrpath)
res = pd.DataFrame(out28)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [40]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [41]:
out29 = process_data(alr_list[28001:29001], alrpath)
res = pd.DataFrame(out29)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [42]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [43]:
out30 = process_data(alr_list[29001:30001], alrpath)
res = pd.DataFrame(out30)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [44]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [45]:
out31 = process_data(alr_list[30001:31001], alrpath)
res = pd.DataFrame(out31)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [46]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [47]:
out32 = process_data(alr_list[31001:32001], alrpath)
res = pd.DataFrame(out32)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [48]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [49]:
out33 = process_data(alr_list[32001:33001], alrpath)
res = pd.DataFrame(out33)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [50]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [51]:
out34 = process_data(alr_list[33001:34001], alrpath)
res = pd.DataFrame(out34)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [52]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [53]:
out35 = process_data(alr_list[34001:35001], alrpath)
res = pd.DataFrame(out35)

1000
100 out of 1000 done
200 out of 1000 done
300 out of 1000 done
400 out of 1000 done
500 out of 1000 done
600 out of 1000 done
700 out of 1000 done
800 out of 1000 done
900 out of 1000 done
1000 out of 1000 done


In [54]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [55]:
out36 = process_data(alr_list[35001:], alrpath) #35299 images
res = pd.DataFrame(out36)

298
100 out of 298 done
200 out of 298 done


In [56]:
with open('allreFeatures.csv', 'a') as f:
        res.to_csv(f, header = False)

In [ ]:
allrdata = open('allreFeatures.csv', 'r')
datadf = pd.DataFrame(allrdata) #slow, data almost 1GB

In [62]:
datadf.shape #dimensions off?

(30298, 1)

In [64]:
allrdata1k = open('processed/allreFeatures1001.csv', 'r')
datadf1k = pd.DataFrame(allrdata1k)

In [65]:
datadf1k.info  #looks ok?

<bound method DataFrame.info of                                                       0
0     ,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,1...
1     0,0.0,0.0,0.3309098482131958,0.021202772855758...
2     1,0.0,0.0,0.0,0.0,0.0,0.0,5.261847019195557,0....
3     2,0.0,0.0,0.0,0.0,0.0,0.0,2.614837408065796,0....
4     3,0.0,0.0,0.0,0.0,3.464846134185791,0.0,0.3073...
...                                                 ...
997   996,0.0,0.0,0.0,0.0,0.0,0.26589658856391907,0....
998   997,0.0,0.0,1.2459115982055664,0.0,0.0,0.0,2.3...
999   998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3....
1000  999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.55414533...
1001  1000,0.0,0.0,0.0,0.0,0.09837108850479126,0.0,0...

[1002 rows x 1 columns]>

In [66]:
datadf1k.shape  #why not 1002 x 4096? separation eror when reading csv?

(1002, 1)

----

In [18]:
# def progressbar(it, prefix="", size=60, file=sys.stdout):
#     count = len(it)
#     def show(j):
#         x = int(size*j/count)
#         file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
#         file.flush()        
#     show(0)
#     for i, item in enumerate(it):
#         yield item
#         show(i+1)
#     file.write("\n")
#     file.flush()
    
    #progressbar really slowing things down! print out iterations in processing ftn instead

In [19]:
# for i in progressbar(range(20), "Computing: ", 40):

# #out1 = process_data(imglist,directory)
#     out1 = process_data(bbc_list[:100], bbcpath)
#     res = pd.DataFrame(out1)
    
    #local:
    #~0.66s per image, expect ~1min for 100 images, ~24min for 2k images? 
    #already took 2mins for 1/20 = 5 images, although right now I'm appending 100 images to dataframe. Memory pressure looks ok though
    #took 22mins for 100 images, not proportional? 
    
    #AWS also slower than expected? 1min for 5 images (twice as fast as local, consistent with prev)
    
    #activated tensorflow environment, still taking 1min for first 5 images

KeyboardInterrupt: 

In [31]:
# if file has already started it starts on next image, otherwise, at the beginning
list_names = !ls
if 'Features.csv' in list_names:
    df = pd.read_csv('Features.csv')
    shoe_num_start = df.shape[0]
else:
    shoe_num_start = 1
shoe_num_start

1

In [32]:
savetopath = 'processed/'

#append processing results to csv:
if shoe_num_start==1:
    res.to_csv(savetopath + 'bbcFeatures100.csv')
else:
    with open(savetopath + 'bbcFeatures100.csv', 'a') as f:
        res.to_csv(f, header = False)
shoe_num_start+=val
shoe_num_start = min(shoe_num_start,len(shoe_number)) #to start where it left off

NameError: name 'val' is not defined

In [ ]:
val = 10

while shoe_num_start<=len(bbc_list):
# for i in range(shoe_num_start,shoe_num_start+val):#range(1,int(np.ceil(len(shoe_number)/val))):
    start = shoe_num_start
    end = min(start+val,len(bbc_list))
    print(start,end)
    out1 = process_data(bbc_list[:20], bbcpath)
    res = pd.DataFrame(out1)
#   res['shoe_number'] = pd.DataFrame(list(range(start,end)))
#  res.set_index('shoe_number', inplace=True)

    if shoe_num_start==1:
        res.to_csv('Features.csv')
    else:
        with open('Features.csv', 'a') as f:
            res.to_csv(f, header = False)
    shoe_num_start+=val
    shoe_num_start = min(shoe_num_start,len(shoe_number)) #to start where it left off

In [80]:
#or skip for now? images not numbered

# val = 10

# while shoe_num_start<=len(shoe_number):
# # for i in range(shoe_num_start,shoe_num_start+val):#range(1,int(np.ceil(len(shoe_number)/val))):
#     start = shoe_num_start
#     end = min(start+val,len(shoe_number))
#     print(start,end)
#     out1 = process_data(start, end)
#     res = pd.DataFrame(out1)
#     res['shoe_number'] = pd.DataFrame(list(range(start,end)))
#     res.set_index('shoe_number', inplace=True)

#     if shoe_num_start==1:
#         res.to_csv('Features.csv')
#     else:
#         with open('Features.csv', 'a') as f:
#             res.to_csv(f, header = False)
#     shoe_num_start+=val #to start where it left off
#     shoe_num_start = min(shoe_num_start,len(shoe_number))  #*this comparison is to get edge cases  
                                                            #- if num =12, val=10, 10,20 can go over and miss final (12),
                                                            #compare and choose smaller of the two
    
    #workaround for not having an image number?